<a href="https://colab.research.google.com/github/D7Knightwalker/AudioVideoTranscribe/blob/main/AudioTranscriber_Phascal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title <h1><b>⬆️**Klik "Run All" Diatas**⬆️</b></i></h1>
# @markdown <p style="text-align: right;">Setelah Selesai menggunakan, Jangan lupa Clear All Output (Edit > Clear all Output)<br>Direkomendasikan untuk menggunakan file berformat: MP3 dan M4A agar ukuran upload tidak teralu besar dan proses tidak berat. Atau <b>Gunakan Link Google Drive!</b><br><br><h2><b>Cukup 🏃RUN ALL ➡️ UPLOAD ☁️ lalu tunggu Selesai!</h2></b><br><p>⚠️Jika muncul <b>Warning: This notebook was not authored by Google</b> cukup <b>Run Anyway</b></p><i> Versi: 3.1 - 30 Nov 2025 <br> Tekan <b>Show code</b> untuk melihat apa yang baru!


# New Update 3.2:
# 1. Test Github


# New Update 3.1:
# 1. Bisa Transkrip via Upload Video/Audio atau
# 2. Metode OCR Baru menggunakan RapidOCR. Agar OCR Lebih Cepat

# New Update 3.0:
# 1. Upload Audio/Video!
# 2. Smart Splitting Audio.
# Apabila 10 detik sebelum durasi Akhir chunks (200s),
# Program akan mencari jeda suara. lalu melakukan cut audio dari jeda suara.
# Metode ini akan meningkatkan keakuratan transkripsi!
# 3. File Transkripsi dilengkapi dengan waktu
# 4. Menambahkan beberapa deskripsi untuk meningkatkan kualitas pengguanaan
# 5. Menambahkan fitur suara! Jika proses selesai, maka akan ada suara!
# 6. Jika menekan Cancel Upload, Maka Cell lain tidak akan muncul Error
# 7. Install Libraries 60s > 120s saat pertama kali dijalankan 😔
# 8. <Percobaan> OCR untuk Video


# New Update 2.5:
# 1. Durasi 1 Chunk Audio 60s > 200s


In [ ]:
%%capture
# @title <center><h3><b>Install Libraries</b><br><i>Untuk Pertama kali akan memakan waktu lebih lama</br>Sekitar 5 Menit</i></h3></center><br>

# 1. UNINSTALL LIBRARY LAMA/BAWAAN (Sangat Penting!)

# 2. INSTALL ONNXRUNTIME KHUSUS CUDA 12
# Ini adalah perintah kunci agar Colab T4 terbaca
!pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

# 3. INSTALL WRAPPER & DEPENDENCIES LAIN
!pip install rapidocr-onnxruntime
!pip install pydub SpeechRecognition silence moviepy tqdm opencv-python-headless gdown

# 3. Import Libraries
import os
import shutil
import time
from tqdm.auto import tqdm
import math
import cv2
import numpy as np

from pydub import AudioSegment
import speech_recognition as sr
from pydub import silence
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import files
from IPython.display import display, HTML, Javascript
from google.colab import output
from moviepy.editor import VideoFileClip
import difflib
import onnxruntime

def cprint(text):
    width = shutil.get_terminal_size().columns
    print(text.center(width))
def play_sound2():
    audio_base64 = "..."

    js_code = f"""
      var audio = new Audio('{audio_base64}');
      audio.play();
    """
    output.eval_js(js_code)

def play_sound():
    audio_base64 = "..."

    js_code = f"""
      var audio = new Audio('{audio_base64}');
      audio.play();
    """
    output.eval_js(js_code)

SUPPORTED_AUDIO = ['.m4a', '.mp3', '.wav', '.ogg', '.opus']
SUPPORTED_VIDEO = ['.mp4', '.mov', '.mkv', '.avi', '.webm']


In [ ]:
# @title <center><h3><b>⬇️UPLOAD FILE DISINI⬇️</b></h3><p>Silakan **pilih/masukan link** file audio/video anda<br>Direkomendasikan upload file AUDIO<br>Kecepatan Upload: Ukuran <b>50MB = 3 Menit</b>. Tergantung Kecepatan Internet</p></center>

from google.colab import files
import gdown

# --- INPUT LINK GOOGLE DRIVE ---
play_sound2() # Memutar suara (dari kode anda sebelumnya)
print("🔗 Masukkan Link Google Drive (Public/Anyone with link).")
print("   Atau tekan [ENTER] langsung jika ingin Upload Manual dari komputer.")
gdrive_url = input(">> Paste Link disini: ").strip()

original_file_path = None

# --- LOGIKA PEMILIHAN ---
if gdrive_url:
    # === OPSI 1: DOWNLOAD DARI GOOGLE DRIVE ===
    print(f"\n🚀 Mendeteksi Link Google Drive...")

    # Buat folder sementara agar rapi
    os.makedirs('downloads', exist_ok=True)

    try:
        # Gdown akan otomatis mendownload file dan memberikan nama aslinya
        # fuzzy=True membantu mendeteksi ID file dari berbagai format link drive
        output_file = gdown.download(gdrive_url, output='downloads/', quiet=False, fuzzy=True)

        if output_file:
            original_file_path = output_file
            print(f"\n✅ Sukses Download dari Drive: {original_file_path}")
        else:
            print("\n❌ Gagal Download. Pastikan Link benar dan aksesnya 'Anyone with the link' (Publik).")

    except Exception as e:
        print(f"\n❌ Terjadi Error saat download: {e}")

else:
    # === OPSI 2: UPLOAD MANUAL ===
    print("\n📤 Masuk ke Mode Upload Manual...")
    uploaded = files.upload()

    if not uploaded:
        print("\n⛔ Proses dihentikan oleh pengguna (Cancel Upload).")
    else:
        original_file_path = list(uploaded.keys())[0]
        print(f"📁 File diterima: {original_file_path}")

# --- VALIDASI AKHIR ---
if original_file_path:
    # Pastikan variabel global tersimpan untuk cell selanjutnya
    file_path = original_file_path
else:
    if 'original_file_path' in globals(): del original_file_path
    if 'file_path' in globals(): del file_path

In [ ]:
# @title <center><h3><b>Opsi Tambahan</p></center>

lakukan_ocr_pada_video = False # @param {type:"boolean"}
# @markdown *PERCOBAAN: <Belum Akurat> Centang jika ingin membaca teks di layar (khusus file Video). Proses ini sangat lama bisa 1 jam lebih tergantung ukuran file*

# Reset variabel global agar tidak tercampur data lama
if 'ocr_text_list' in globals(): del ocr_text_list
ocr_text_list = []


In [ ]:
# @title <center><h3><b>Inisialisasi Mesin OCR🚀</b></h3></center>
import re
import cv2
import os
import shutil
import onnxruntime as ort
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from rapidocr_onnxruntime import RapidOCR

# --- KONFIGURASI ---
OCR_INTERVAL_DETIK = 25
IMAGE_SIMILARITY_THRESHOLD = 0.90
MAX_OCR_WORKERS = 25
MAX_CAPTURE_WORKERS = 10 # Worker khusus untuk menyimpan gambar (IO Bound)

# --- CEK GPU & INIT OCR ---
available_providers = ort.get_available_providers()
if 'CUDAExecutionProvider' in available_providers:
    print("✅ GPU T4 Terdeteksi! OCR Engine siap ngebut.")
    ocr_engine = RapidOCR()
else:
    print("⚠️ GPU tidak terdeteksi. Menggunakan CPU.")
    ocr_engine = RapidOCR()

def calculate_image_similarity(img1, img2):
    """Membandingkan 2 gambar (Histogram)"""
    try:
        # Input dari OpenCV sudah BGR, jadi convert BGR ke HSV
        hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
        hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
        hist1 = cv2.calcHist([hsv1], [0, 1], None, [180, 256], [0, 180, 0, 256])
        hist2 = cv2.calcHist([hsv2], [0, 1], None, [180, 256], [0, 180, 0, 256])
        cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
        cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)
        return cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    except: return 0

def worker_ocr_task(image_path):
    """Worker OCR (GPU/CPU)"""
    try:
        result, _ = ocr_engine(image_path)
        full_text = ""
        if result:
            txts = [line[1] for line in result]
            full_text = " ".join(txts)

        text_clean = re.sub(r'\s+', ' ', full_text).strip()
        has_real_content = re.search(r'[a-zA-Z0-9]{3,}', text_clean)

        if has_real_content and len(text_clean) > 5:
            return (image_path, text_clean)
        return None
    except: return None

def save_frame_worker(frame, path):
    """Worker khusus untuk menyimpan gambar agar main thread tidak block"""
    try:
        cv2.imwrite(path, frame)
    except: pass

def process_video_ocr_and_capture(video_path, interval=OCR_INTERVAL_DETIK, similarity_threshold=IMAGE_SIMILARITY_THRESHOLD):
    # Gunakan OpenCV VideoCapture (Lebih cepat dari MoviePy)
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("❌ Gagal membuka video.")
        return []

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    base_name = os.path.splitext(os.path.basename(video_path))[0]
    capture_folder = os.path.join("video_captures", base_name)
    if os.path.exists(capture_folder): shutil.rmtree(capture_folder)
    os.makedirs(capture_folder, exist_ok=True)

    # Hitung frame mana saja yang harus diambil (Lompat per interval)
    frame_interval = int(fps * interval)
    target_indices = list(range(0, total_frames, frame_interval))

    print(f"📸 PHASE 1: Turbo Scanning Video ({int(duration)}s)...")

    saved_images_list = []
    prev_frame_small = None

    # Executor khusus untuk menyimpan file (agar tidak menunggu disk writing)
    save_executor = ThreadPoolExecutor(max_workers=MAX_CAPTURE_WORKERS)

    for frame_idx in tqdm(target_indices, unit="chk", desc="🎞️ Fast Capture"):
        # Lompat langsung ke frame tujuan (Fast Seeking)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()

        if not ret: break

        should_save = False

        # Resize kecil untuk perbandingan super cepat
        # Frame dari OpenCV sudah BGR, tidak perlu convert warna
        small_curr = cv2.resize(frame, (64, 64))

        if prev_frame_small is None:
            should_save = True
        else:
            sim_score = calculate_image_similarity(small_curr, prev_frame_small)
            if sim_score < similarity_threshold:
                should_save = True

        if should_save:
            # Hitung Timestamp
            seconds = frame_idx / fps
            m, s = int(seconds // 60), int(seconds % 60)
            h = int(seconds // 3600)
            timestamp_name = f"{h:02d}_{m:02d}_{s:02d}.jpg"
            save_path = os.path.join(capture_folder, timestamp_name)

            saved_images_list.append(save_path)

            # Submit tugas simpan ke thread lain (Non-blocking)
            save_executor.submit(save_frame_worker, frame.copy(), save_path)

            prev_frame_small = small_curr

    cap.release()
    save_executor.shutdown(wait=True) # Pastikan semua file tersimpan sebelum lanjut

    if not saved_images_list:
        print("⚠️ Tidak ada gambar unik yang ditemukan.")
        return []

    print(f"\n🚀 PHASE 2: Menjalankan RapidOCR (GPU) pada {len(saved_images_list)} gambar...")
    final_results_path = []

    with ThreadPoolExecutor(max_workers=MAX_OCR_WORKERS) as executor:
        futures = {executor.submit(worker_ocr_task, img_path): img_path for img_path in saved_images_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="📖 Extracting"):
            result = future.result()
            if result: final_results_path.append(result[0])
            else:
                try: os.remove(futures[future])
                except: pass

    final_results_path.sort()
    print(f"✅ Selesai! Disimpan {len(final_results_path)} gambar berisi teks.")
    return final_results_path

In [ ]:
# @title <center><h3><b>Cek & Konversi Video ke Audio 🎞️➡️🎵</b></h3></center>
# @markdown <p style="text-align: right;">Jalan Proses:<br>1. Extract Audio<br>2. Capture/Screenshot per 10 Detik (5 Menit untuk 80 Frame)<br>3. Hasil Capture akan diproses OCR (5 Menit untuk 60 Frames)</p>
import subprocess
import os

def process_media_file(path, enable_ocr):
    global saved_capture_paths

    filename, ext = os.path.splitext(path)
    ext = ext.lower()

    audio_path_result = path

    if ext in SUPPORTED_VIDEO:
        print(f"🎞️ Terdeteksi VIDEO ({ext}).")

        # A. Ekstrak Audio dengan FFmpeg (SUPER CEPAT)
        try:
            print("   -> Mengekstrak audio track (FFmpeg Direct)...")
            audio_output = f"{filename}_extracted.wav"

            # Perintah FFmpeg:
            # -i : Input file
            # -vn : Video No (Jangan proses videonya, buang visualnya) -> Ini bikin cepat!
            # -acodec pcm_s16le : Codec audio WAV standar
            # -ar 44100 : Sample rate audio (standar)
            # -ac 2 : Stereo
            # -y : Overwrite jika file sudah ada
            # -threads 0 : Gunakan semua core CPU

            command = f'ffmpeg -y -i "{path}" -vn -acodec pcm_s16le -ar 44100 -ac 2 -threads 0 "{audio_output}" -loglevel error'

            # Jalankan perintah di terminal system
            subprocess.run(command, shell=True, check=True)

            audio_path_result = audio_output
            print(f"   ✅ Audio berhasil diekstrak: {audio_output}")

        except subprocess.CalledProcessError as e:
            print(f"   ❌ Gagal ekstrak audio dengan FFmpeg, mencoba fallback MoviePy...")
            # Fallback ke MoviePy jika FFmpeg gagal (Jaga-jaga)
            try:
                video = VideoFileClip(path)
                video.audio.write_audiofile(audio_output, logger=None)
                video.close()
                audio_path_result = audio_output
            except Exception as e2:
                print(f"   ❌ Gagal Total: {e2}")
                return path

        # B. Jalankan OCR & Capture (Jika user memilih YA)
        if enable_ocr:
            # Tidak perlu 'global' lagi di sini
            saved_capture_paths = process_video_ocr_and_capture(original_file_path)
        else:
            print("   ℹ️ OCR & Capture dilewati (opsi tidak dicentang).")
            # Tidak perlu 'global' lagi di sini
            saved_capture_paths = []

    # 2. JIKA AUDIO
    elif ext in SUPPORTED_AUDIO:
        print(f"🎵 Terdeteksi AUDIO ({ext}). OCR tidak diperlukan.")
        saved_capture_paths = []

    else:
        print(f"⚠️ Format {ext} tidak umum. Mencoba proses langsung.")
        saved_capture_paths = []

    return audio_path_result

# EKSEKUSI UTAMA
if 'original_file_path' in globals():
    # Perbaikan: Buat variabel kosong HANYA jika belum ada, agar eksekusi di dalam fungsi aman.
    if 'saved_capture_paths' not in globals(): globals()['saved_capture_paths'] = []

    file_path = process_media_file(original_file_path, lakukan_ocr_pada_video)
else:
    print("⚠️ Harap upload file terlebih dahulu.")

In [ ]:
# @title <center><h3><b>Memotong Audio Menjadi ~200 Detik 🔉</b></h3></center>

def convert_to_chunks(file_path, chunk_length_ms=200000):
    # --- 1. Cek Format & Load Audio ---
    ext = os.path.splitext(file_path)[1].lower()
    if ext not in SUPPORTED_AUDIO:
        print(f"❌ Format tidak didukung: {ext}")
        return []

    print("⏳ Memuat audio (sedang memproses waveform)...")
    audio = AudioSegment.from_file(file_path)

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_folder = os.path.join("audio_chunks", base_name)
    os.makedirs(output_folder, exist_ok=True)

    # --- 2. Konfigurasi Smart Split ---
    silence_thresh = audio.dBFS - 16

    chunk_paths = []
    start_time = 0
    audio_len = len(audio)
    chunk_idx = 0

    print(f"🎯 Memproses: {base_name} | Durasi Total: {audio_len/1000:.1f}s")

    # Progress bar manual karena jumlah chunk dinamis
    pbar = tqdm(total=audio_len, unit="ms", desc="✂️ Smart Splitting")

    # --- 3. Loop Pemotongan Dinamis ---
    while start_time < audio_len:
        end_time = start_time + chunk_length_ms

        # Jika sisa audio kurang dari batas maksimal, ambil sisanya
        if end_time >= audio_len:
            split_point = audio_len
        else:
            # Cari hening di 'jendela pencarian'
            search_window_len = 15000 # 15 detik
            search_start = max(start_time, end_time - search_window_len)
            segment_to_search = audio[search_start:end_time]

            # Deteksi silence
            silences = silence.detect_silence(
                segment_to_search,
                min_silence_len=500,
                silence_thresh=silence_thresh
            )

            if silences:
                last_silence_start = silences[-1][0]
                split_point = search_start + last_silence_start
                split_point += 100
            else:
                split_point = end_time

        # --- 4. Ekspor Chunk ---
        chunk = audio[start_time:split_point]
        chunk_name = os.path.join(output_folder, f"chunk{chunk_idx}.wav")
        chunk.export(chunk_name, format="wav")
        chunk_paths.append(chunk_name)

        # Update progress bar
        pbar.update(split_point - start_time)

        # Geser start_time TEPAT ke titik potong (tidak ada yang terlewati)
        start_time = split_point
        chunk_idx += 1

    pbar.close()
    print(f"✅ Selesai! Terbagi menjadi {len(chunk_paths)} bagian.")
    return chunk_paths

if 'file_path' in globals():
    chunk_paths = convert_to_chunks(file_path)
else:
    print(f"⚠️ Proses telah di cancel karena cancel upload")

In [ ]:
# @title <center><h3><b>Memproses Transkrip Audio 📝</b></h3></center>

def _pick_best_text(results):
    if isinstance(results, str): return results, None
    if not results or 'alternative' not in results or not results['alternative']: return "", None
    best, best_conf = None, -1.0
    for alt in results['alternative']:
        txt = alt.get('transcript', '') or ''
        conf = alt.get('confidence', None)
        if conf is not None and conf > best_conf:
            best, best_conf = txt, conf
        elif best is None and txt:
            best = txt
    return (best, best_conf) if best else ("", None)

# def _transcribe_with_langs(recognizer, audio, langs=('id-ID', 'en-US')):
def _transcribe_with_langs(recognizer, audio, langs=['id-ID']):
    best_text, best_lang, best_conf = "", None, -1.0

    if isinstance(langs, str):
        langs = [langs]

    for lang in langs:
        try:
            raw = recognizer.recognize_google(audio, language=lang, show_all=True)
            text, conf = _pick_best_text(raw)
            if not text:
                text = recognizer.recognize_google(audio, language=lang)
                conf = None
        except (sr.UnknownValueError, sr.RequestError):
            text, conf = "", None

        score = (conf if conf is not None else 0.0) + (len(text) / 1000.0)
        if text and score > best_conf:
            best_text, best_lang, best_conf = text, lang, score
    return best_text, best_lang, best_conf

def recognize_chunks_parallel(chunk_paths, langs=['id-ID'], max_workers=50):
# def recognize_chunks_parallel(chunk_paths, langs=('id-ID','en-US'), max_workers=50):
    recognizer = sr.Recognizer()
    transcripts = [None] * len(chunk_paths)

    def recognize_chunk(i, path):
        try:
            with sr.AudioFile(path) as source:
                audio = recognizer.record(source)
            text, lang_used, _ = _transcribe_with_langs(recognizer, audio, langs=langs)
            return i, text if lang_used else "[Tidak terdengar]"
        except Exception as e:
            return i, f"[Error: {e}]"

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(recognize_chunk, i, path) for i, path in enumerate(chunk_paths)]
        for future in tqdm(as_completed(futures), total=len(futures), desc="📝 Transcribing", unit="chunk"):
            i, text = future.result()
            transcripts[i] = text

    # PERUBAHAN DISINI: Mengembalikan LIST, bukan string yang sudah digabung
    return transcripts

if 'file_path' in globals():
  start = time.time()
  # Simpan hasil dalam list transcripts_list
  transcripts_list = recognize_chunks_parallel(chunk_paths)
  end = time.time()
  print(f"\n✅ Transkripsi selesai dalam {int(end - start)} detik.")
else:
    print(f"⚠️ Proses telah di cancel karena cancel upload")

In [ ]:
# @title <center><h3><b>Proses Selesai!<br>Menunggu File Download📝</b></h3></center>
import os
import shutil
import zipfile
from google.colab import files
from pydub import AudioSegment

# --- FUNGSI UTAMA (Fix Error NameError) ---
def clean_transcription(text):
    if not text: return ""
    text = text.strip()
    return text[0].upper() + text[1:] if len(text) > 0 else text

def format_time(seconds):
    m, s = int(seconds // 60), int(seconds % 60)
    return f"{m:02d}:{s:02d}"

if 'file_path' in globals():
    base_filename = os.path.splitext(os.path.basename(file_path))[0]
    clean_name = base_filename.replace("_extracted", "")
    output_dir_name = f"OUTPUT_{clean_name}"

    # Reset Output Folder
    if os.path.exists(output_dir_name): shutil.rmtree(output_dir_name)
    os.makedirs(output_dir_name, exist_ok=True)

    # --- 1. PROSES TEXT ---
    full_text_time = []
    current_time = 0.0

    # Fallback jika transcript kosong
    if 'transcripts_list' not in globals(): transcripts_list = []
    if 'chunk_paths' not in globals(): chunk_paths = []

    print("⏳ Menyusun file teks...")

    cleaned_transcripts = [clean_transcription(t) for t in transcripts_list]

    for path, text in zip(chunk_paths, cleaned_transcripts):
        if not text: continue
        try:
            chunk_audio = AudioSegment.from_file(path)
            dur = chunk_audio.duration_seconds
            paragraph = f"[{format_time(current_time)} - {format_time(current_time + dur)}]\n{text}\n"
            full_text_time.append(paragraph)
            current_time += dur
        except: continue

    # Simpan TXT
    txt_path = os.path.join(output_dir_name, f"{clean_name}_TRANSKRIP.txt")
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(full_text_time))

    # --- 2. PROSES GAMBAR OCR ---
    ref_path = original_file_path if 'original_file_path' in globals() else file_path
    capture_source = os.path.join("video_captures", os.path.splitext(os.path.basename(ref_path))[0])

    if os.path.exists(capture_source):
        capture_dest = os.path.join(output_dir_name, "CAPTURED_FRAMES")
        shutil.move(capture_source, capture_dest)
        print(f"📦 Mengarsipkan {len(os.listdir(capture_dest))} gambar OCR...")
    else:
        print("📦 Hanya file teks (Tidak ada hasil OCR).")

    # --- 3. ZIP & DOWNLOAD ---
    zip_name = f"{output_dir_name}.zip"
    shutil.make_archive(output_dir_name, 'zip', os.getcwd(), output_dir_name)

    print(f"✅ Download dimulai: {zip_name}")
    files.download(zip_name)

    if 'play_sound' in globals(): play_sound()
else:
    print("⚠️ Tidak ada file untuk diproses.")